In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competicao-dois-ic/train.csv
/kaggle/input/competicao-dois-ic/test.csv


In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pandas as pd
from itertools import zip_longest

treino = pd.read_csv('/kaggle/input/competicao-dois-ic/train.csv')
teste = pd.read_csv('/kaggle/input/competicao-dois-ic/test.csv')

#print(treino)

textos = treino['text']
classe = treino['misogynous']

x = pd.concat([textos], axis=1)
x = x.dropna()
y = classe
aloneWords = textos.str.lower().str.split() #colocar todas as palavras em minúsculo
#print(aloneWords) 

dicionario = set()

for lista in aloneWords:
  dicionario.update(lista) #monta um dicionario pra cada palavra do dataset

#print(dicionario)
print(len(dicionario))

wordPosition = dict(zip(dicionario, range(len(dicionario))))
#print(wordPosition)

def vetorizeCountWords(texto, wordPosition):
  vetor = [0] * len(wordPosition)
  for word in texto:
    if word in wordPosition:
      position = wordPosition[word]
      vetor[position] += 1
  return vetor

vetorText = [vetorizeCountWords(texto, wordPosition) for texto in aloneWords]

#print(vetorText)

24558


In [6]:
from sklearn.neural_network import MLPClassifier

from sklearn.datasets import make_classification


X_train, X_test, y_train, y_test = train_test_split(vetorText,y, test_size=0.20, random_state=0)

#dt = DecisionTreeClassifier(max_depth=None, criterion='entropy', splitter='random', min_impurity_decrease= 0.000000001, max_leaf_nodes= 100, random_state= 0)
#dt = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
dt = MLPClassifier(solver='sgd', alpha=1e-5, random_state=0, activation = 'tanh', learning_rate= 'adaptive')

dt.fit(X_train, y_train)

conf = confusion_matrix(y_test, dt.predict(X_test))
accuracy = accuracy_score(y_test, dt.predict(X_test))

#clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

#accuracy = predition.score(X_test, y_test)

print(conf)
print(accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[[587 169]
 [189 555]]
0.7613333333333333


In [ ]:
textos1 = teste['text']

'''
x1 = pd.concat([textos1], axis=1) #concatena em uma só tabela os valores númericos e categoricos pré-tratados
x1 = x1.dropna()
'''

aloneWords1 = textos1.str.lower().str.split() #colocar todas as palavras em minúsculo

'''
dicionario1 = set()
for lista in aloneWords1:
  dicionario1.update(lista) #monta um dicionario pra cada palavra do dataset
'''

wordPosition1 = dict(zip(dicionario, range(len(dicionario))))

def vetorizeCountWords(texto1, wordPosition1):
  vetor = [0] * len(wordPosition1)
  for word in texto1:
    if word in wordPosition1:
      position = wordPosition1[word]
      vetor[position] += 1
  return vetor

vetorText1 = [vetorizeCountWords(texto1, wordPosition1) for texto1 in aloneWords1]

tab = dt.predict(vetorText1)

submission = pd.DataFrame.from_records(zip_longest(teste['id'], tab), columns=['id', 'misogynous'])
submission.set_index('id').to_csv('submission.csv')
print(submission)